In [ ]:
import ipyleaflet as lf

In [ ]:
import json
from operator import itemgetter

from traitlets import (CaselessStrEnum, Unicode, Tuple, List, Bool, CFloat, Enum,
                       Float, CInt, Int, Instance, Dict, Bytes, Any, Union, Undefined)

import ipywidgets as widgets
from ipywidgets import Color
from ipywidgets.widgets.trait_types import TypedTuple, ByteMemoryView
from ipywidgets.widgets.widget_link import Link


In [ ]:
classes = {"map": lf.Map, "map-style": lf.MapStyle, "search-control": lf.SearchControl, "legend-control": lf.LegendControl,
           "attribution-control": lf.AttributionControl, "scale-control": lf.ScaleControl, "zoom-control": lf.ZoomControl,
           "draw-control": lf.DrawControl, "split-map-control": lf.SplitMapControl, "measure-control": lf.MeasureControl,
           "layers-control": lf.LayersControl, "full-screen-control": lf.FullScreenControl, "widget-control": lf.WidgetControl,
           "control": lf.Control, "choropleth": lf.Choropleth, "geo-data": lf.GeoJSON, "geo-json": lf.GeoJSON,
           "feature-group": lf.FeatureGroup, "layer-group": lf.LayerGroup, "marker-cluster": lf.MarkerCluster, "circle": lf.Circle,
           "circle-marker": lf.CircleMarker, "rectangle": lf.Rectangle, "polygon": lf.Polygon, "polyline": lf.Polyline,
           "ant-path": lf.AntPath, "path": lf.Path, "vector-layer": lf.VectorLayer, "vector-tile-layer": lf.VectorTileLayer,
           "heatmap": lf.Heatmap, "video-overlay": lf.VideoOverlay, "image-overlay": lf.ImageOverlay, "wms-layer": lf.WMSLayer,
           "local-tile-layer": lf.TileLayer, "tile-layer": lf.TileLayer, "raster-layer": lf.RasterLayer, "popup": lf.Popup,
           "marker": lf.Marker, "awesome-icon": lf.AwesomeIcon, "icon": lf.Icon, "ui-layer": lf.UILayer, "layer": lf.Layer}

Copied from https://github.com/jupyter-widgets/ipywidgets/blob/master/packages/schema/generate-spec.py

In [ ]:
def trait_type(trait, widget_list):
    attributes = {}
    if isinstance(trait, (CaselessStrEnum, Enum)):
        w_type = 'string'
        attributes['enum'] = trait.values
    elif isinstance(trait, Unicode):
        w_type = 'string'
    elif isinstance(trait, (Tuple, List)):
        w_type = 'array'
    elif isinstance(trait, TypedTuple):
        w_type = 'array'
        attributes['items'] = trait_type(trait._trait, widget_list)
    elif isinstance(trait, Bool):
        w_type = 'bool'
    elif isinstance(trait, (CFloat, Float)):
        w_type = 'float'
    elif isinstance(trait, (CInt, Int)):
        w_type = 'int'
    elif isinstance(trait, Color):
        w_type = 'color'
    elif isinstance(trait, Dict):
        w_type = 'object'
    elif isinstance(trait, (Bytes, ByteMemoryView)):
        w_type = 'bytes'
    elif isinstance(trait, Instance) and issubclass(trait.klass,
                                                     widgets.Widget):
        w_type = 'reference'
        attributes['widget'] = trait.klass.__name__
        # ADD the widget to this documenting list
        if (trait.klass not in [i[1] for i in widget_list]
                and trait.klass is not widgets.Widget):
            widget_list.append((trait.klass.__name__, trait.klass))
    elif isinstance(trait, Union):
        w_type = 'union'
        attributes['types'] = [trait_type(t, widget_list) for t in trait.trait_types]
    elif isinstance(trait, Any):
        # In our case, these all happen to be values that are converted to
        # strings
        w_type = 'label'
    else:
        w_type = trait.__class__.__name__
    attributes['type'] = w_type
    if trait.allow_none:
        attributes['allow_none'] = True
    return attributes


In [ ]:
def jsdefault(trait):
    if isinstance(trait, Instance):
        default = trait.make_dynamic_default()
        if issubclass(trait.klass, widgets.Widget):
            return 'reference to new instance'
    else:
        default = trait.default_value
        if isinstance(default, bytes) or isinstance(default, memoryview) or isinstance(default, type(Undefined)):
            default = trait.default_value_repr()
    return default

In [ ]:
def jsonify(identifier, widget, widget_list):
    n = identifier 
    attributes = []
    for name, trait in widget.traits(sync=True).items():
        if name == '_view_count':
            # don't document this since it is totally experimental at this point
            continue

        attribute = dict(
            name=name,
            help=trait.help or '',
            default=jsdefault(trait)
        )
        attribute.update(trait_type(trait, widget_list))
        attributes.append(attribute)

    return dict(name=n, attributes=attributes)

In [ ]:
def create_spec(widget_list):
    widget_data = []
    for widget_name, widget_cls in widget_list:
        if issubclass(widget_cls, Link):
            widget = widget_cls((widgets.IntSlider(), 'value'),
                                (widgets.IntSlider(), 'value'))
        elif issubclass(widget_cls, (widgets.SelectionRangeSlider,
                                     widgets.SelectionSlider)):
            widget = widget_cls(options=[1])
        elif issubclass (widget_cls, lf.LegendControl):
            widget = widget_cls({})
        elif issubclass (widget_cls, lf.SearchControl):
            widget = widget_cls(marker=lf.Marker())
        elif issubclass (widget_cls, lf.WidgetControl):
            widget = widget_cls(widget=widgets.DOMWidget())                
        else:
            widget = widget_cls()

        widget_data.append(jsonify(widget_name, widget, widget_list))
    return widget_data

In [ ]:
sort_cl = sorted([[k, classes[k]] for k in classes])

In [ ]:
specs = create_spec(sort_cl)

In [ ]:
with open('../../resources/leaflet-schema.json', "w") as file:
    json.dump(specs, file, sort_keys=True, indent=2, separators=(',', ': '))

In [ ]:
bm = lf.basemaps

In [ ]:
with open('../../resources/basemaps.json', "w") as file:
    json.dump(bm, file, sort_keys=True, indent=2, separators=(',', ': '))